In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [5]:
st = pd.read_csv("stocks.csv")

In [6]:
st.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-02-07,150.639999,155.229996,150.639999,154.649994,154.414230,83322600
1,AAPL,2023-02-08,153.880005,154.580002,151.169998,151.919998,151.688400,64120100
2,AAPL,2023-02-09,153.779999,154.330002,150.419998,150.869995,150.639999,56007100
3,AAPL,2023-02-10,149.460007,151.339996,149.220001,151.009995,151.009995,57450700
4,AAPL,2023-02-13,150.949997,154.259995,150.919998,153.850006,153.850006,62199000


# Descriptive Statisstic

In [7]:
DS = st.groupby('Ticker')['Close'].describe()
DS

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,62.0,158.240645,7.360485,145.309998,152.077499,158.055000,165.162506,173.570007
GOOG,62.0,100.631532,6.279464,89.349998,94.702501,102.759998,105.962503,109.459999
MSFT,62.0,275.039839,17.676231,246.270004,258.742500,275.810013,287.217506,310.649994
NFLX,62.0,327.614677,18.554419,292.760010,315.672493,325.600006,338.899994,366.829987


In [8]:
DS = st.groupby('Ticker')['Open'].describe()
DS

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,62.0,157.779839,7.224608,144.380005,151.489998,158.400002,164.702503,170.979996
GOOG,62.0,100.381919,6.197598,89.540001,94.532499,102.680000,105.859999,107.800003
MSFT,62.0,274.735969,17.324808,246.550003,257.410004,277.110001,285.825005,307.760010
NFLX,62.0,328.110643,18.467142,287.339996,317.137497,325.649994,340.674995,372.410004


In [9]:
DS = st.groupby('Ticker')['High'].describe()
DS

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,62.0,159.473710,7.167176,146.710007,153.545002,159.869995,165.750004,174.300003
GOOG,62.0,101.792516,6.229136,90.129997,95.962498,103.885002,107.088249,109.629997
MSFT,62.0,277.764356,17.435444,250.929993,260.357498,280.835007,289.044991,311.970001
NFLX,62.0,332.648065,18.086059,297.450012,322.909996,331.255005,344.670006,373.829987


# Time series Analysis

In [12]:
st['Date'] = pd.to_datetime(st['Date'])
pivotdata = st.pivot(index='Date', columns='Ticker', values='Close')

In [14]:
fig = make_subplots(rows=1, cols=1)

In [16]:
for column in pivotdata.columns:
    fig.add_trace(
        go.Scatter(x=pivotdata.index, y=pivotdata[column], name=column),
        row=1,col=1
    )

In [17]:
fig.update_layout(
    title_text = 'Time Series of closing Prices',
    xaxis_title = 'Date',
    yaxis_title = 'Closing Price',
    legend_title = 'Ticker',
    showlegend=True
)

In [18]:
fig.show()

# Volatility Analysis

In [19]:
volatility = pivotdata.std().sort_values(ascending=False)

In [20]:
fig = px.bar(volatility,
             x= volatility.index,
             y = volatility.values,
             labels = {'y' : 'Standard Deviation', 'x' :'Ticker'},
             title='Volatility of Closing Prices(Standard Deviation)'
             )

In [21]:
fig.show()

# Correlation Analysis

In [22]:
correlation_matrix = pivotdata.corr()
fig  = go.Figure(data=go.Heatmap(
    z = correlation_matrix,
    x = correlation_matrix.columns,
    y = correlation_matrix.columns,
    colorscale = 'blues',
    colorbar=dict(title='Correlation')
))

In [23]:
fig.update_layout(
    title='Correlation Matrix of closing Prices',
    xaxis_title='Ticker',
    yaxis_title='Ticker'
)

# Comparative Analysis

In [30]:
percentage_change = ((pivotdata.iloc[-1] - pivotdata.iloc[0]) /pivotdata.iloc[0]) * 100

In [31]:
fig = px.bar(percentage_change,
             x=percentage_change.index,
             y=percentage_change.values,
             labels={'y':'Percentage Change (%)', 'x':'Ticker'},
             title='Percentage Change in closing Prices')

In [32]:
fig.show()

# Daily Risk vs Return Analysis

In [34]:
daily_returns = pivotdata.pct_change().dropna()


In [36]:
avg_daily_return = daily_returns.mean()
risk = daily_returns.std()

In [37]:
risk_return_df = pd.DataFrame({'Risk': risk, 'Average Daily Return' : avg_daily_return})

fig = go.Figure()

In [38]:
fig.add_trace(go.Scatter(
    x=risk_return_df['Risk'],
    y=risk_return_df['Average Daily Return'],
    mode = 'markers+text',
    text=risk_return_df.index,
    textposition = 'top center',
    marker=dict(size=10)
))

In [39]:
fig.update_layout(
    title='Risk vs. Return Analysis',
    xaxis_title='Risk (standard deviation)',
    yaxis_title = 'Average Daily Return',
    showlegend=False
)